# Spectral viewer demo with a notebook  
(Modified version of the notebook available on prospect's github repo)   
All the desi packages along with bokeh and numba should be installed

In [ ]:
import os, sys
import numpy as np
from astropy.table import Table
from pathlib import Path
import desispec
import importlib

# uncomment this and replace with proper path if the redrock template environment variable is not found
# os.environ["RR_TEMPLATE_DIR"] = "/opt/astrosoft/redrock-templates"

# Change the path below based on where you have downloaded prospect
sys.path.append("/home/bid13/astrosoft/prospect/py")
from prospect import utilities,plotframes

# importlib.reload(utilities)
# importlib.reload(plotframes)

In [ ]:
# Path to the directory where files are saved
data_path = Path("/data/bid13/desi/70003")

I had concatenated all the observed fits files to get a master file and used the table below to select appropriate target IDs. But you can use any mini-SV zbest and co-add file you like and do your own selection of subsets.

In [ ]:
table = Table.read(data_path/"zbest-70003-20200219-sv0_lrg_truth_added.fits")
print(len(table))
targets = table[(table["Z"]<1)&(table["DELTACHI2"]<300)]["TARGETID"]

In [ ]:
# Read the files from the mini-SV

zbfile = data_path/"zbest-0-70003-20200219.fits" #"selected_zbest.fits"
specfile = data_path/"coadd-0-70003-20200219.fits" #"selected_coadd.fits"

spectra = desispec.io.read_spectra(specfile)

zbest = Table.read(zbfile, 'ZBEST')


In [ ]:
print(str(spectra.num_spectra())+" spectra in total")

#If you want to do the step below i.e. select spectra based on targetIDs you should provide a fake column
spectra.fibermap["NIGHT"]=np.ones(len(spectra.fibermap))

# Select a subset based on the target
sp_single_exp = spectra.select(targets=targets)
print(str(sp_single_exp.num_spectra())+" spectra selected")

#### Spectral viewer  
DO NOT load more than 100 spectra into the viewer as it might overload your web browser.


In [ ]:
plotframes.plotspectra(sp_single_exp, zcatalog=zbest, notebook=True, with_coaddcam=False, with_thumb_tab=False)